In [22]:
#%store -r df

In [46]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
import pandas as pd

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

In [47]:
# tells matplotlib to embed plots within the notebook
#%matplotlib inline

In [48]:
os.chdir('C:\\Users\\belincoln\\repos\\BudgetPredict')
# Set working directory to the data folder so you can correctly read in the csv files
%cd data

C:\Users\belincoln\repos\BudgetPredict\data


In [49]:
# Read data from DHS Contracts
df = pd.read_csv('FY2019_070_Contracts_Full_20200110_1.csv', header = 0, usecols = ['contract_transaction_unique_key', 
                        'federal_action_obligation','total_dollars_obligated', 'base_and_exercised_options_value', 
                        'current_total_value_of_award', 'base_and_all_options_value','potential_total_value_of_award'],
                 dtype = {'contract_transaction_unique_key':'str','federal_action_obligation': 'float',
                        'total_dollars_obligated': 'float', 'base_and_exercised_options_value': 'float', 
                        'current_total_value_of_award': 'float', 'base_and_all_options_value': 'float',
                        'potential_total_value_of_award': 'float'})

In [50]:
# Create 3 new features for analysis
df['Percent awarded over potential total awarded'] = df['current_total_value_of_award'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over potential total value of award'] = df['total_dollars_obligated'] / df['potential_total_value_of_award']
df['Percent Cumulatively Obligated over total value already awarded'] = df['total_dollars_obligated'] / df['current_total_value_of_award']

# Create Indicator Variable
df['Indicator'] = df['federal_action_obligation']<-1000

# set index to each transaction key
df.set_index('contract_transaction_unique_key', inplace = True)

In [51]:
df.drop('federal_action_obligation', axis =1, inplace = True)
df = df.fillna(0)

# Beginning of Regression Analysis

In [52]:
df.columns

Index(['total_dollars_obligated', 'base_and_exercised_options_value',
       'current_total_value_of_award', 'base_and_all_options_value',
       'potential_total_value_of_award',
       'Percent awarded over potential total awarded',
       'Percent Cumulatively Obligated over potential total value of award',
       'Percent Cumulatively Obligated over total value already awarded',
       'Indicator'],
      dtype='object')

In [53]:
df.shape

(66533, 9)

In [54]:
df.dtypes
    

total_dollars_obligated                                               float64
base_and_exercised_options_value                                      float64
current_total_value_of_award                                          float64
base_and_all_options_value                                            float64
potential_total_value_of_award                                        float64
Percent awarded over potential total awarded                          float64
Percent Cumulatively Obligated over potential total value of award    float64
Percent Cumulatively Obligated over total value already awarded       float64
Indicator                                                                bool
dtype: object

In [55]:
X, y =  df.iloc[:,:-1], df.loc[:,'Indicator']

In [33]:
def  featureNormalize(X):
    """
    Normalizes the features in X. returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when working with
    learning algorithms.
    
    Parameters
    ----------
    X : array_like
        The dataset of shape (m x n).
    
    Returns
    -------
    X_norm : array_like
        The normalized dataset of shape (m x n).
    
    Instructions
    ------------
    First, for each feature dimension, compute the mean of the feature
    and subtract it from the dataset, storing the mean value in mu. 
    Next, compute the  standard deviation of each feature and divide
    each feature by it's standard deviation, storing the standard deviation 
    in sigma. 
    
    Note that X is a matrix where each column is a feature and each row is
    an example. You needto perform the normalization separately for each feature. 
    
    Hint
    ----
    You might find the 'np.mean' and 'np.std' functions useful.
    """
    # You need to set these values correctly
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    # =========================== YOUR CODE HERE =====================
    mu = np.mean(X,axis=0)
    sigma = np.std(X,axis=0)
    X_norm = (X-mu)/sigma
    # ================================================================
    return X_norm, mu, sigma
print(np.mean(X,axis=0))


total_dollars_obligated                                               4.287900e+06
base_and_exercised_options_value                                      2.652147e+05
current_total_value_of_award                                          4.360394e+06
base_and_all_options_value                                            2.585394e+06
potential_total_value_of_award                                        2.365486e+07
Percent awarded over potential total awarded                          8.497815e-01
Percent Cumulatively Obligated over potential total value of award    8.404629e-01
Percent Cumulatively Obligated over total value already awarded       9.223830e-01
dtype: float64


In [34]:
X_norm, mu, sigma = featureNormalize(X)

<a id="section1"></a>

#### Sigmoid function

the logistic regression hypothesis is defined as:

$$ h_\theta(x) = g(\theta^T x)$$

where function $g$ is the sigmoid function. The sigmoid function is defined as: 

$$g(z) = \frac{1}{1+e^{-z}}$$.

 For large positive values of `x`, the sigmoid should be close to 1, while for large negative values, the sigmoid should be close to 0. Evaluating `sigmoid(0)` should give you exactly 0.5. 
 
 This is used for classification models (rather than regresssion models).
 
<a id="sigmoid"></a>

In [56]:
def sigmoid(z):
    """
    Compute sigmoid function given the input z.
    
    Parameters
    ----------
    z : array_like
        The input to the sigmoid function. This can be a 1-D vector 
        or a 2-D matrix. 
    
    Returns
    -------
    g : array_like
        The computed sigmoid function. g has the same shape as z, since
        the sigmoid is computed element-wise on z.
        
    Instructions
    ------------
    Compute the sigmoid of each value of z (z can be a matrix, vector or scalar).
    """
    # convert input to a numpy array
    z = np.array(z)
    
    # You need to return the following variables correctly 
    g = np.zeros(z.shape)

    temp = 1 + np.power(np.e,-z)
    g = 1 / temp
    

    return g

In [57]:
# Setup the data matrix appropriately, and add ones for the intercept term
m, n = X.shape
# Add intercept term to X
X = np.concatenate([np.ones((m, 1)), X_norm], axis=1)

In [58]:
# convert y to np.array of 0s and 1s
y = np.array(y.astype(int))

Develop a function `costFunction` to return the cost and gradient. Recall that the cost function in logistic regression is

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \left[ -y^{(i)} \log\left(h_\theta\left( x^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - h_\theta\left( x^{(i)} \right) \right) \right]$$

and the gradient (derivative) of the cost is a vector of the same length as $\theta$ where the $j^{th}$
element (for $j = 0, 1, \cdots , n$) is defined as follows:

$$ \frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^m \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} $$

Note that while this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $h_\theta(x)$.


In [61]:
def costFunction(theta, X, y):
    """
    Compute cost and gradient for logistic regression. 
    
    Parameters
    ----------
    theta : array_like
        The parameters for logistic regression. This a vector
        of shape (n+1, ).
    
    X : array_like
        The input dataset of shape (m x n+1) where m is the total number
        of data points and n is the number of features. We assume the 
        intercept has already been added to the input.
    
    y : arra_like
        Labels for the input. This is a vector of shape (m, ).
    
    Returns
    -------
    J : float
        The computed value for the cost function. 
    
    grad : array_like
        A vector of shape (n+1, ) which is the gradient of the cost
        function with respect to theta, at the current values of theta.
        
    Instructions
    ------------
    Compute the cost of a particular choice of theta. You should set J to 
    the cost. Compute the partial derivatives and set grad to the partial
    derivatives of the cost w.r.t. each parameter in theta.
    """
    # Initialize some useful values
    m = y.size  # number of training examples

    # You need to return the following variables correctly 
    J = 0
    grad = np.zeros(theta.shape)

    # ====================== YOUR CODE HERE ======================

    h = sigmoid(np.dot(X,theta))
    temp = -y*np.log(h) - (1-y)*np.log(1-h)
    J = (1/m)*np.sum(temp)
    grad = (1/m)*np.dot(X.T,(h-y))
    # =============================================================
    return J, grad

In [39]:
# Initialize fitting parameters
initial_theta = np.zeros(n+1)
print(initial_theta)
cost, grad = costFunction(initial_theta, X, y)
print(cost)

print('Cost at initial theta (zeros):' + str(cost))


print('Gradient at initial theta (zeros):')
print(grad)


# Compute and display cost and gradient with non-zero theta
test_theta = np.array([-24, 0.2, 0.2,3,4,5,6,7,8])
cost, grad = costFunction(test_theta, X, y)

print('Cost at test theta: {:.3f}'.format(cost))


print('Gradient at test theta: ' + str(grad))


[0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.6931471805599454
Cost at initial theta (zeros):0.6931471805599454
Gradient at initial theta (zeros):
[ 0.4110216  -0.00184109  0.00936506 -0.00177696  0.00311307  0.00327162
  0.00702852  0.0053589   0.0151578 ]
Cost at test theta: nan
Gradient at test theta: [-0.08588547  0.03642699  0.01514494  0.03641646  0.0297934   0.03406495
  0.00388522  0.00236244  0.01139037]


C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in multiply


In [62]:
# set options for optimize.minimize
options= {'maxiter': 400}

# see documention for scipy's optimize.minimize  for description about
# the different parameters
# The function returns an object `OptimizeResult`
# We use truncated Newton algorithm for optimization which is 
# equivalent to MATLAB's fminunc
# See https://stackoverflow.com/questions/18801002/fminunc-alternate-in-numpy
res = optimize.minimize(costFunction,
                        initial_theta,
                        (X, y),
                        jac=True,
                        method='TNC',
                        options=options)

# the fun property of `OptimizeResult` object returns
# the value of costFunction at optimized theta
cost = res.fun

# the optimized theta is in the x property
theta = res.x

# Print theta to screen
print('Cost at theta found by optimize.minimize: ' + str(cost))


print('theta:')
print(theta)


C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: overflow encountered in power
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: divide by zero encountered in log
C:\Users\belincoln\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in multiply


Cost at theta found by optimize.minimize: 0.2983502743615043
theta:
[-2.18734618  0.01533018 -0.15916793  0.01424393 -0.04214222 -0.05694922
 -0.02841107  0.01745807 -0.15920108]


In [41]:
# This produces a scatter plot with each of the features, 
# but does not cover each permutation of features
# 10 min runtime, I should rethink this

'''
features = len(df.columns) - 2
for i in range(features):
    fig = pyplot.figure()
    mask  = y == 1
    pyplot.plot(X[mask].iloc[:,i], X[mask].iloc[:, i+1], 'k*', lw=2, ms=10)
    pyplot.plot(X[~mask].iloc[:, i], X[~mask].iloc[:, i+1], 'ko', mfc='y', ms=8, mec='k', mew=1)
   
    # add axes labels
    pyplot.xlabel(df.columns[i])
    pyplot.ylabel(df.columns[i+1])
    pyplot.legend(['Sweep', 'Not a Sweep'])
    pass
'''

"\nfeatures = len(df.columns) - 2\nfor i in range(features):\n    fig = pyplot.figure()\n    mask  = y == 1\n    pyplot.plot(X[mask].iloc[:,i], X[mask].iloc[:, i+1], 'k*', lw=2, ms=10)\n    pyplot.plot(X[~mask].iloc[:, i], X[~mask].iloc[:, i+1], 'ko', mfc='y', ms=8, mec='k', mew=1)\n   \n    # add axes labels\n    pyplot.xlabel(df.columns[i])\n    pyplot.ylabel(df.columns[i+1])\n    pyplot.legend(['Sweep', 'Not a Sweep'])\n    pass\n"